In [1]:
%run C:/__PYP/__COMMON/ugenejupyterlib/firstcell.ipynb
%run C:/__PYP/__COMMON/ugenejupyterlib/datacell.ipynb
    
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

import pandas as pd
import numpy as np

import plotly.io as pio
pio.renderers.default = "browser"

@
        # platform.node()='6700k-1080' 
        # datetime.now()=datetime.datetime(2020, 12, 15, 21, 8, 55, 412789) 
        # sys.version='3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]' 
        


In [50]:
import chardet    
rawdata = open(filepath, 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
print(charenc)

ISO-8859-1


In [2]:
pwd = Path(rf'C:\Users\eliu\Desktop\isaac\logs')

ldfs = []

list_of_pathobjects = sorted(pwd.glob('*.csv'))
for f in list_of_pathobjects:
    ldfs.append(pd.read_csv(
        f, 
        na_values=np.nan, 
        encoding='ISO-8859-1',
    )) # raw
    

df = pd.concat(ldfs, axis=0)
df.info()

dfraw = df.copy()

ValueError: No objects to concatenate

In [4]:
df = dfraw.copy()
df['t'] = df['Date'] + ' ' + df['Time']

df.t = pd.to_datetime(df.t)

df.t[ pd.isnull(df.t) ]

df = df.set_index('t')
df = df.drop(columns='Date Time'.split())


dfcat = df.copy()

Series([], Name: t, dtype: datetime64[ns])

In [51]:
# [↓]

import pandas as pd
import numpy as np
df = pd.DataFrame(columns=list('abcd'), data=np.random.randn(20, 4),
                  index=pd.date_range("2020-10-10", periods=20, freq="1min"))


# for something quick and dirty, px is great. 
# [1] It can interpret kwarg "y" very flexibly, gave it colname, or actual col, all good. 
# [2] px will also automatically assign coln to trace name, makes it easy to "selector" when fig.update_traces()
import plotly.express as px
fig = px.scatter(df, x=df.index, y=df.columns)
fig = px.scatter(df, x=df.index, y=['c', df['b']])

# with some fig.update_traces() that is common to 
fig = fig.update_traces(
    selector=dict(name="c"),
    marker=dict(color="rgba(0, 0, 0, .3)", size=22, symbol="201"),
    mode="lines+markers",
)


# [THE DRAWBACKs]
# can NOT support multiaxis, can't specify during ploting, 
fig = fig.add_trace(go.Scatter(
    x=df.index, y=df['a'], yaxis='y2'
))
fig = fig.update_layout(
    xaxis=dict(
        type="date",
        domain=[0, 1], # display    
        rangeslider=dict(
            autorange=True,
            visible=True,
        ),
    ),
    yaxis1=dict(
        anchor='free',
        #rangemode='tozero',
        linecolor='green', linewidth=2,
        showgrid=False, zeroline=False,
    ),
    yaxis2=dict(
        anchor="free",
        
        overlaying="y", 
            # [!] crucially important, otherwise stuff on y axis can not hover mode
        
        side="left",
        position=0.95,
        linecolor='red', linewidth=2,
        showgrid=False, zeroline=False,
    )
)
fig.show()



In [33]:
df = dfcat.copy().head(1000).iloc[:, 0:11]


# the automatic binning routine.
stat = pd.DataFrame(index = df.columns, data = [df[cn].describe() for cn in df.columns])

stat

def _absrange(row, num):
    return abs(row['min'])<num and abs(row['max'])<num  
def foo(row):
    output = None
    for x in [5, 10, 50, 100]:
        if _absrange(row, x):
            output = x
            break
    return output

binning = stat.apply(foo, axis=1)
binning.name = 'bin'

binning['Membrane Pressure (Bar)']=5
binning

,count,mean,std,min,25%,50%,75%,max
Membrane Pressure (Bar),1000.0,0.761387,1.641695e+00,0.14979,0.154010,0.16051,0.176705,5.3418
Membrane Pressure (Bar),1000.0,0.450794,8.244135e-01,0.13065,0.134690,0.14136,0.157340,2.8701
Feed Temperature °C,1000.0,18.757791,1.404228e+00,15.53800,17.650000,19.38450,19.926500,20.4160
Reject Water Flow (l/min),1000.0,0.450425,1.185306e+00,0.00170,0.008140,0.02443,0.036990,3.8919
Permeate Flow (l/min),1000.0,1.883359,5.128897e+00,-0.02545,-0.018240,-0.00127,0.011450,16.6030
Raw Water Turbidity (NTU),1000.0,10.809783,1.478317e+00,7.67420,9.866725,11.06400,11.999000,13.0900
Permeate Turbidity (NTU),1000.0,0.011849,7.728259e-04,0.01011,0.011370,0.01190,0.012260,0.0165
Treated Water Chorine (ppm),1000.0,1.810454,2.438838e-01,1.26240,1.598125,1.89140,2.055500,2.0730
Treated Water Chorine PH,1000.0,8.816580,4.217992e-02,8.68200,8.778550,8.81440,8.854550,8.9059
Distribution System Water Flow (l/min),1000.0,6.081535,2.942135e+00,0.79660,4.952700,5.56350,6.248100,40.5880


Membrane Pressure (Bar)                    5
Membrane Pressure  (Bar)                   5
Feed Temperature °C                       50
Reject Water Flow (l/min)                  5
Permeate Flow (l/min)                     50
Raw Water Turbidity (NTU)                 50
Permeate Turbidity (NTU)                   5
Treated Water Chorine (ppm)                5
Treated Water Chorine PH                  10
Distribution System Water Flow (l/min)    50
Dose Factor                                5
Name: bin, dtype: int64

In [47]:
import random
df = dfcat.copy()
fig = go.Figure()

for coln in df.columns[:11]:
    fig=fig.add_trace(go.Scattergl(
        
        # BASICs
        x=df.index, y=df[coln], name=coln,
        mode='lines', connectgaps=True, # js is tricter? connectgaps=1 doesn't work

#         opacity=0.75,

        
        # HOVER
        text = [coln]*len(df.index), # unfortunately, can not access kwarg "name"
            # for %{var} only x, y, text and customdata is accessable 
        hovertemplate =
            '%{y} %{text}'
            '<extra></extra>',
        
    ))

yaxistitles = dict(zip(
    'y1 y2 y3'.split(), ['']*3
))

from plotly.validators.scatter.marker import SymbolValidator
raw_symbols = SymbolValidator().values

for cn in df.columns[:11]:
    if binning[cn] == 5:
        yaxistitles['y1'] = yaxistitles['y1'] + cn + '; '
        fig = fig.update_traces(
            selector=dict(   name=cn,    ),
            mode="lines+markers",
                # "lines", "markers", "lines+markers", "lines+markers+text"

            marker=dict(
                color=f"rgba({random.randint(0, 155)}, 255, {random.randint(0, 155)}, .5)",
                size=7, 
                symbol = random.choice(raw_symbols),
            ),
            yaxis = 'y1',
        )
    elif binning[cn] == 10:
        yaxistitles['y2'] = yaxistitles['y2'] + cn + '; '
        fig = fig.update_traces(
            selector=dict(   name=cn,    ),
            mode="lines+markers",
                # "lines", "markers", "lines+markers", "lines+markers+text"

            marker=dict(
                color=f"rgba(255, {random.randint(0, 155)}, {random.randint(0, 155)}, .5)",
                size=7,  
                symbol = random.choice(raw_symbols),
            ),
            yaxis = 'y2',
        )
    elif binning[cn] == 50:
        yaxistitles['y3'] = yaxistitles['y3'] + cn + '; '
        fig = fig.update_traces(
            selector=dict(   name=cn,    ),
            mode="lines+markers",
                # "lines", "markers", "lines+markers", "lines+markers+text"

            marker=dict(
                color=f"rgba( {random.randint(0, 155)}, {random.randint(0, 155)}, 255, .5)",

                size=7,    
                symbol = random.choice(raw_symbols),
            ),
            yaxis = 'y3',
        )    


fig = fig.update_layout(
    xaxis=dict(
        type="date",
        domain=[0, 1], # display
        range=[
            df.index[0], df.index[ int(len(df.index) / 4) ]
        ], # for range slider and initial data range
        
        rangeslider=dict(
            autorange=True,
            visible=True,
        ),
    ),
    yaxis1=dict(
        title=yaxistitles['y1'], titlefont=dict(color='green'), tickfont=dict(color='green'),
        anchor='free',
        #rangemode='tozero',
        linecolor='green', linewidth=2,
        showgrid=False, zeroline=False,
    ),
    yaxis2=dict(
        title=yaxistitles['y2'], titlefont=dict(color='red'), tickfont=dict(color='red'),
        anchor="free",
        overlaying="y", # [!] crucially important, otherwise stuff on y axis can not hover mode
        side="left",
        position=0.95,
        linecolor='red', linewidth=2,
        showgrid=False, zeroline=False,
    ),
    yaxis3=dict(
        title=yaxistitles['y3'], titlefont=dict(color='blue'), tickfont=dict(color='blue'),
        anchor="free",
        overlaying="y", # [!] crucially important, otherwise stuff on y axis can not hover mode
        side="left",
        position=1,
        linecolor='blue', linewidth=2,
        showgrid=False, zeroline=False,
    ),
)

                  
fig.show()


#[!!!] how to use dictionary of properties, and the underscore syntax

In [28]:
l="""
B	business day frequency
C	custom business day frequency
D	calendar day frequency
W	weekly frequency
M	month end frequency
SM	semi-month end frequency (15th and end of month)
BM	business month end frequency
CBM	custom business month end frequency
MS	month start frequency
SMS	semi-month start frequency (1st and 15th)
BMS	business month start frequency
CBMS	custom business month start frequency
Q	quarter end frequency
BQ	business quarter end frequency
QS	quarter start frequency
BQS	business quarter start frequency
A, Y	year end frequency
BA, BY	business year end frequency
AS, YS	year start frequency
BAS, BYS	business year start frequency
BH	business hour frequency
H	hourly frequency
T, min	minutely frequency
S	secondly frequency
L, ms	milliseconds
U, us	microseconds
N	nanoseconds
""".strip().split('\n')

def foo(s):
    s = s.split('\t')
    return f"{s[0]:<10} {s[1]}"

[print(foo(x)) for x in l]

B          business day frequency
C          custom business day frequency
D          calendar day frequency
W          weekly frequency
M          month end frequency
SM         semi-month end frequency (15th and end of month)
BM         business month end frequency
CBM        custom business month end frequency
MS         month start frequency
SMS        semi-month start frequency (1st and 15th)
BMS        business month start frequency
CBMS       custom business month start frequency
Q          quarter end frequency
BQ         business quarter end frequency
QS         quarter start frequency
BQS        business quarter start frequency
A, Y       year end frequency
BA, BY     business year end frequency
AS, YS     year start frequency
BAS, BYS   business year start frequency
BH         business hour frequency
H          hourly frequency
T, min     minutely frequency
S          secondly frequency
L, ms      milliseconds
U, us      microseconds
N          nanoseconds


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [25]:
df

,a,b,c
2020-01-01 00:00:00,101.764052,20.007860,0.978738
2020-01-01 00:01:00,102.240893,93.377900,-0.977278
2020-01-01 00:02:00,100.950088,-7.567860,-0.103219
2020-01-01 00:03:00,100.410599,7.202179,1.454274
2020-01-01 00:04:00,100.761038,6.083751,0.443863
2020-01-01 00:05:00,100.333674,74.703954,-0.205158
2020-01-01 00:06:00,100.313068,-42.704787,-2.552990
2020-01-01 00:07:00,100.653619,43.221810,-0.742165
2020-01-01 00:08:00,102.269755,-72.718284,0.045759
2020-01-01 00:09:00,99.812816,76.638961,1.469359


In [47]:
printdf(df.head())

[df] None:
+---------------------+------------+------------+------------+
|                     |   processA |   processB |   processC |
|---------------------+------------+------------+------------|
| 2020-01-01 00:00:00 |    101.764 |   20.0079  |   0.978738 |
| 2020-01-01 00:01:00 |    102.241 |   93.3779  |  -0.977278 |
| 2020-01-01 00:02:00 |    100.95  |   -7.56786 |  -0.103219 |
| 2020-01-01 00:03:00 |    100.411 |    7.20218 |   1.45427  |
| 2020-01-01 00:04:00 |    100.761 |    6.08375 |   0.443863 |
+---------------------+------------+------------+------------+



In [48]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = "browser"

np.random.seed(0)
rowcounts = 10
df = pd.DataFrame(
    index = pd.date_range(start=pd.Timestamp('2020-01-01'), freq = 'min', periods=rowcounts, ),
    data = np.random.randn(rowcounts, 3), columns=['process'+ x for x in list('ABC')],
)
df.processA += 100
df.processB *= 50

idx = pd.date_range(start=pd.Timestamp('2020-01-01'), freq = '30S', periods=rowcounts*2, )
ef = pd.DataFrame([
    dict(alm="alarm A something something has alarmed", Start=idx[1], Finish=idx[3], almid="alm_id A"),
    dict(alm="alarm B something else happened", Start=idx[3], Finish=idx[5], almid="alm_id B"),
    dict(alm="alarm A something something has alarmed", Start=idx[4], Finish=idx[-1], almid="alm_id A")
])

"""
df==
+---------------------+------------+------------+------------+
|                     |   processA |   processB |   processC |
|---------------------+------------+------------+------------|
| 2020-01-01 00:00:00 |    101.764 |   20.0079  |   0.978738 |
| 2020-01-01 00:01:00 |    102.241 |   93.3779  |  -0.977278 |
| 2020-01-01 00:02:00 |    100.95  |   -7.56786 |  -0.103219 |
| 2020-01-01 00:03:00 |    100.411 |    7.20218 |   1.45427  |
| 2020-01-01 00:04:00 |    100.761 |    6.08375 |   0.443863 |
+---------------------+------------+------------+------------+

ef==
+----+-----------------------------------------+---------------------+---------------------+----------+
|    | alm                                     | Start               | Finish              | almid    |
|----+-----------------------------------------+---------------------+---------------------+----------|
|  0 | alarm A something something has alarmed | 2020-01-01 00:00:30 | 2020-01-01 00:01:30 | alm_id A |
|  1 | alarm B something else happened         | 2020-01-01 00:01:30 | 2020-01-01 00:02:30 | alm_id B |
|  2 | alarm A something something has alarmed | 2020-01-01 00:02:00 | 2020-01-01 00:09:30 | alm_id A |
+----+-----------------------------------------+---------------------+---------------------+----------+
"""

fig = px.timeline(
    ef, x_start="Start", x_end="Finish", y="alm", color="almid",
    color_discrete_map={
        "alm_id A": 'rgba(222, 111, 222, .2)',
        "alm_id B": 'rgba(111, 222, 222, .2)',
    }
)

myfont=dict(
    color="red",
    family="monospace",
    size=14,
)
for c in df.columns:
    fig = fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[c],
            name = c,
            yaxis = 'y2',
            hovertemplate = '%{y} <extra></extra>',
        ))
    
fig = fig.update_layout(
    xaxis=dict(
        type="date",
        domain=[0.15,1],
        range = [idx[2], idx[10]],
        rangeslider=dict( # terrible performance
            autorange=True,
            visible=True,
        ),
    ),
    yaxis=dict(
        title="alarms",
        titlefont=myfont,
        tickfont=myfont,
    ),
    yaxis2=dict(
        title="processes",
        anchor="free",
        overlaying="y",
        side="left",
        position=0.05
    ),
)

fig = fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
fig.show()

'\ndf==\n+---------------------+------------+------------+------------+\n|                     |   processA |   processB |   processC |\n|---------------------+------------+------------+------------|\n| 2020-01-01 00:00:00 |    101.764 |   20.0079  |   0.978738 |\n| 2020-01-01 00:01:00 |    102.241 |   93.3779  |  -0.977278 |\n| 2020-01-01 00:02:00 |    100.95  |   -7.56786 |  -0.103219 |\n| 2020-01-01 00:03:00 |    100.411 |    7.20218 |   1.45427  |\n| 2020-01-01 00:04:00 |    100.761 |    6.08375 |   0.443863 |\n+---------------------+------------+------------+------------+\n\nef==\n+----+-----------------------------------------+---------------------+---------------------+----------+\n|    | alm                                     | Start               | Finish              | almid    |\n|----+-----------------------------------------+---------------------+---------------------+----------|\n|  0 | alarm A something something has alarmed | 2020-01-01 00:00:30 | 2020-01-01 00:01:30 |

In [82]:
ccc=\
"""
hover text
the key problem is that it can only use limited amount of variables.

[+] hovertemplate, can use 'text' and 'customdata' kwarg
"""
########################################################################################
# plotly stuff
########################################################################################


l_xvals = df.index.to_timestamp()
    #[!] df.index at the moment is time period, which doens't work well with plotly 
   #  "Object of type Period is not JSON serializable"

df_meta = pd.merge(left=preplot, right=ref, how='left', on='id')
df_meta['start'], df_meta['end'] = df_meta['start'].dt.strftime('%Y-%b'), df_meta['end'].dt.strftime('%Y-%b')


fig= go.Figure()

for coln in df.columns:

    meta = df_meta.loc[df_meta['id']==coln, :].to_dict('records')[0]
    fillcolor, linecolor, linewidth = 0,0,1

    if len(meta['problem']) > 0:
        fillcolor = 'rgba(0, 0, 0, 0)'
        linecolor = '#cc11cc'
        linewidth = 0.1
    elif not "In-Progress" in meta['status']:
        fillcolor = 'rgba(99, 99, 99, .1)'
        linecolor = '#666'
        linewidth = 0.5  
    else:
        fillcolor = getcolor(meta['priority'])
        linecolor = None
        linewidth = 1   
        
    fig = fig.add_trace(go.Bar(
        name=coln, x=l_xvals, y=df[coln],  
        
        customdata = [meta]*len(df.index),
        
        hovertemplate =
        '<b>[%{customdata.id}]  %{customdata.problem}</b>   <i>%{customdata.status}</i>' + '<br>'
        '<b>%{customdata.title}</b>' +'<br>'
        '<i>PRIORITY</i>: %{customdata.priority} | ' + '<i>monthlyHrs</i>: %{y} | '
        '%{customdata.percent}% of %{customdata.hours} <i>hrs total</i> <br>'
        '%{customdata.start} ~ %{customdata.end} <br>'
        '☻ %{customdata.sact}' +'<br>'
        '☻ %{customdata.scmt0}' +'<br>'
        '%{customdata.scmt1}' +'<br>'           
        '<extra></extra>',

        texttemplate = '%{customdata.id}',            
        textposition="auto",
        
        textfont=dict(
            color=linecolor,
        ),
        
        marker=dict(
            color = fillcolor,
            line=dict(
                color=linecolor,
                width=linewidth,
            ),
        ),
        
    ))

fig = fig.add_shape(type="line",
    x0=l_xvals[0], y0=125, x1=l_xvals[-1], y1=125,
    line=dict(
        color="rgba(255, 22, 99, 0.4)",
        width=3,
        dash="dot",
    )
)
fig = fig.update_layout(barmode='stack')
fig = fig.update_layout(
    title= meta['who'],
    
    hoverlabel=dict(
        font_size=16,
        font_family="monospace"
    ),
    xaxis=dict( #[!!!!!!]
        title="time",
        
        tickformat="%Y-%b",
#         nticks=12,
        tick0 = l_xvals[0],

        dtick = "M1",
        
        type="date",
        # type="category",
        range=[l_xvals[0] , l_xvals[13] if len(l_xvals)>13 else l_xvals[-1]
              ],
#         rangeslider=dict( # terrible performance
#             autorange=False,
#             visible=True,
#         ),
    ),    
)
pio.show(fig)